In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "12345"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style={'height': '10%', 'width': '10%'}),style={'textAlign': 'center'}),
    html.Div(html.Center(html.B(html.H1('Daniel Tipton SNHU CS-340-T4211 Dashboard')))),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    # I added in radio items for selection of the filtering
    html.Hr(),
    html.Div(dcc.RadioItems(
                id='radio-items',
                options = [
                    {'label': 'Water Rescue', 'value': '1'},
                    {'label': 'Mountain Rescue', 'value': '2'},
                    {'label': 'Disaster Rescue', 'value': '3'},
                    {'label': 'Reset', 'value': '4'},

                    ],
                value = '',
                labelStyle={'display': 'inline-block'}
    )), 


    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable = False,
        filter_action ="native",
        sort_action ="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        #Aligned the Text Left for better veiwing
        style_cell={'textAlign': 'left'},
        

    ),
   
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    # changed this to make it work with my update function
@app.callback(Output('datatable-id','data'),
    [Input('radio-items', 'value')])

def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if (int(filter_type) == 4):
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog"}))
    # use higher number of button clicks to determine filter type, can you think of a better way ? ....
    elif (int(filter_type) == 1):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},"sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})))
    elif (int(filter_type) == 2):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["German Shepherd","Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})))
    elif (int(filter_type) == 3):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}})))
       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]
    
    dff = pd.DataFrame.from_dict(viewData)
    values = dff.groupby(['breed']).size()
    names = dff.breed.unique()
    
    return [
        dcc.Graph(            
            figure = px.pie(dff, values=values, names=names)
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]
    

app